# Recognizing Handwritten Digits

1. MINST dataset
2. 解析度 28 * 28 總共784個pixels
3. 每個pixels都會以darkness來記錄 (0為white, 1為dark)

In [77]:
import pandas as pd

In [107]:
data = pd.read_csv("MNIST dataset\\mnist.csv", header=None)
print(data.head())
print(data.shape)

   0    1    2    3    4    5    6    7    8    9    ...    775    776   777  \
0    5    0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  ...  0.608  0.609  0.61   
1    4    0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000  0.000  0.00   
2    3    0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000  0.000  0.00   
3    0    0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000  0.000  0.00   
4    2    0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000  0.000  0.00   

     778    779    780    781    782    783    784  
0  0.611  0.612  0.613  0.614  0.615  0.616  0.617  
1  0.000  0.000  0.000  0.000  0.000  0.000  0.000  
2  0.000  0.000  0.000  0.000  0.000  0.000  0.000  
3  0.000  0.000  0.000  0.000  0.000  0.000  0.000  
4  0.000  0.000  0.000  0.000  0.000  0.000  0.000  

[5 rows x 785 columns]
(2001, 785)


In [109]:
X = data.drop(0, axis = 1).to_numpy()
X.shape

(2001, 784)

In [110]:
y_raw = data[0].to_numpy()
print(y_raw[0:10])
print(y_raw.shape)
print(len(y_raw))

[5 4 3 0 2 8 8 1 2 6]
(2001,)
2001


In [139]:
# An example of one-hot encoding
import numpy as np

def dummies(n, labels):
    # The number of image categories
    n_categories = n

    # The unique values of categories in the data
    categories = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    # Initialize ohe_labels as all zeros
    ohe_labels = np.zeros((len(labels), n_categories))

    # Loop over the labels
    for ii in range(len(labels)):
        # Find the location of this label in the categories variable
        jj = np.where(labels[ii] == categories)
        # Set the corresponding zero to one
        ohe_labels[ii,jj] = 1

    return ohe_labels

In [140]:
y = dummies(10, y_raw)

In [141]:
y.shape

(2001, 10)

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=21)

In [115]:
print(X_train.shape)
print(y_train.shape)

(1600, 784)
(1600, 10)


___

# Start training

In [119]:
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [196]:
# Create the model: model
model = Sequential()

# Add the first hidden layer
model.add(Dense(160, activation = 'relu', input_shape = (784,)))

# Add the second hidden layer
model.add(Dense(100, activation = 'relu'))

# Add the third hidden layer
#model.add(Dense(50, activation = 'relu'))

# Add the output layer
model.add(Dense(10, activation = 'softmax'))

# Compile the model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Set up Early Stopping Monitor
early_stopping_monitor = EarlyStopping(patience = 3)

# Fit the model
model.fit(X_train, y_train, validation_split = 0.3, epochs = 20, callbacks = [early_stopping_monitor])

Train on 1120 samples, validate on 480 samples
Epoch 1/20
1120/1120 [==============================] - 0s 149us/step - loss: 22.6325 - accuracy: 0.5420 - val_loss: 7.6986 - val_accuracy: 0.7375
Epoch 2/20
1120/1120 [==============================] - 0s 62us/step - loss: 3.2325 - accuracy: 0.8304 - val_loss: 5.2653 - val_accuracy: 0.7729
Epoch 3/20
1120/1120 [==============================] - 0s 76us/step - loss: 1.2135 - accuracy: 0.9107 - val_loss: 4.9672 - val_accuracy: 0.8250
Epoch 4/20
1120/1120 [==============================] - 0s 76us/step - loss: 0.5713 - accuracy: 0.9482 - val_loss: 4.1610 - val_accuracy: 0.8479
Epoch 5/20
1120/1120 [==============================] - 0s 62us/step - loss: 0.2098 - accuracy: 0.9795 - val_loss: 3.6587 - val_accuracy: 0.8500
Epoch 6/20
1120/1120 [==============================] - 0s 84us/step - loss: 0.1230 - accuracy: 0.9777 - val_loss: 3.7448 - val_accuracy: 0.8438
Epoch 7/20
1120/1120 [==============================] - 0s 76us/step - loss: 0.12

### Save the model

In [197]:
from keras.models import load_model
model.save('Saved_Model/Handwritten_digit_classification.h5')

### Reload the model

In [198]:
my_model = load_model('Saved_Model/Handwritten_digit_classification.h5')

In [199]:
my_model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 160)               125600    
_________________________________________________________________
dense_57 (Dense)             (None, 100)               16100     
_________________________________________________________________
dense_58 (Dense)             (None, 10)                1010      
Total params: 142,710
Trainable params: 142,710
Non-trainable params: 0
_________________________________________________________________


### Prediction

In [200]:
classified_digits = my_model.predict_classes(X_test)
classified_digits

array([6, 0, 3, 2, 0, 2, 8, 9, 1, 0, 9, 0, 8, 3, 9, 6, 4, 7, 4, 3, 1, 3,
       9, 8, 5, 1, 6, 0, 0, 4, 5, 3, 8, 0, 9, 0, 5, 0, 0, 1, 4, 0, 4, 5,
       5, 8, 7, 5, 6, 9, 6, 6, 1, 7, 4, 5, 9, 5, 7, 2, 8, 8, 4, 0, 5, 6,
       6, 6, 9, 6, 3, 9, 1, 9, 0, 7, 7, 9, 8, 1, 7, 2, 0, 0, 8, 1, 0, 6,
       4, 6, 8, 2, 0, 3, 0, 7, 6, 6, 3, 9, 9, 0, 0, 6, 6, 5, 8, 3, 9, 4,
       9, 7, 2, 9, 3, 2, 8, 5, 8, 8, 3, 0, 0, 9, 7, 6, 1, 8, 4, 0, 5, 9,
       4, 0, 5, 5, 8, 6, 6, 8, 9, 1, 2, 1, 8, 2, 7, 0, 4, 8, 2, 6, 0, 9,
       5, 1, 5, 3, 9, 9, 8, 5, 3, 1, 8, 7, 9, 5, 6, 2, 8, 2, 6, 6, 2, 2,
       7, 5, 8, 4, 6, 6, 1, 0, 4, 1, 9, 0, 8, 2, 7, 4, 6, 3, 1, 7, 5, 7,
       4, 4, 7, 8, 7, 2, 6, 5, 8, 5, 0, 8, 6, 9, 9, 2, 3, 3, 6, 5, 1, 8,
       4, 0, 6, 3, 9, 7, 5, 5, 7, 0, 7, 9, 0, 0, 5, 4, 0, 7, 6, 9, 6, 6,
       1, 7, 0, 1, 4, 1, 7, 8, 8, 2, 9, 8, 6, 8, 9, 2, 6, 5, 8, 6, 7, 5,
       1, 8, 9, 1, 4, 1, 7, 6, 3, 2, 1, 7, 0, 1, 3, 1, 7, 1, 9, 9, 2, 5,
       6, 6, 7, 1, 8, 6, 5, 3, 8, 4, 9, 1, 9, 2, 0,

In [201]:
y_pred = dummies(10, classified_digits)
y_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [202]:
number_correct = (y_pred*y_test).sum()
print(number_correct)

331.0


In [211]:
accuracy = number_correct/len(y_pred)
print("The prediction accuracy of the handwritten-digit recognition model is " + str(round(accuracy*100, 3)) + "%")

The prediction accuracy of the handwritten-digit recognition model is 82.544%
